## Imports 

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import joblib
import ast
import os
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
import pandas as pd
import os
import numpy as np
import ast
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
import joblib

from sklearn.utils import resample

C:\Users\pdevi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\pdevi\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Définition de fonctions importantes 

Le **taux de couverture des tags** est calculé en vérifiant combien de tags réels sont correctement prédits au fil des échantillons de test. Pour chaque prédiction, la fonction **`calculate_tag_coverage`** compare les tags réels aux tags prédits et ajoute les tags correspondants dans un ensemble de tags déjà couverts (covered_tags). À la fin de la boucle, le nombre de tags distincts correctement prédits est divisé par le nombre total de tags dans le jeu de données, donnant ainsi le taux de couverture. Plus ce ratio est élevé, plus l'algorithme couvre un large éventail de tags dans ses prédictions.

La **précision** est calculée à l'aide de la fonction **`calculate_accuracy`**, qui vérifie si chaque tag prédit correspond à un sous-ensemble des tags réels. Pour chaque prédiction, si un tag prédit correspond à une sous-chaîne dans un tag réel, il est considéré comme correct. Le taux de précision est ensuite calculé en divisant le nombre de prédictions correctes par le nombre total de tags réels. Pour éviter des valeurs dépassant 1, la précision maximale est limitée à 1. La précision reflète la qualité des prédictions de l'algorithme par rapport aux tags attendus.

Ce code effectue une classification multi-étiquette en traitant des données prétraitées, en filtrant les étiquettes les plus courantes, puis en entraînant plusieurs modèles de machine learning. Voici un résumé en français :

1. **Traitement des données** :
   - Charge les données prétraitées et extrait les étiquettes.
   - Filtre les données selon les 15 et 50 étiquettes les plus fréquentes, créant ainsi deux configurations (`top_15` et `top_50`).

2. **Entraînement et évaluation des modèles** :
   - Plusieurs classificateurs (RandomForest, XGBoost, LogisticRegression, GradientBoosting, AdaBoost) sont entraînés en utilisant une stratégie One-vs-Rest.
   - Une recherche d’hyperparamètres est effectuée avec `GridSearchCV` pour chaque modèle.
   - Les modèles sont évalués sur la base de métriques de précision et de couverture, et les résultats sont enregistrés avec MLflow.

3. **Réduction dimensionnelle** :
   - PCA (Analyse en Composantes Principales) est appliquée pour réduire la dimensionnalité des caractéristiques avant l'entraînement des modèles.

4. **Précision et couverture des étiquettes** :
   - Le script calcule la précision des prédictions d'étiquettes et la couverture pour des nombres variables d'étiquettes prédites, puis génère des graphiques.

5. **Sauvegarde et enregistrement** :
   - Les meilleurs modèles, PCA et les étiquettes binarisées (via `MultiLabelBinarizer`) sont sauvegardés et enregistrés avec MLflow et joblib.

Le code facilite les expériences répétées, l’enregistrement des résultats, ainsi que la visualisation des performances des modèles à travers différentes configurations et ensembles de caractéristiques.

In [3]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
import mlflow
import mlflow.sklearn
import joblib
import os
import matplotlib.pyplot as plt
import ast

# Évaluer en toute sécurité une chaîne de caractères en une liste
def safely_evaluate_list(tag_string):
    try:
        return ast.literal_eval(tag_string)
    except (ValueError, SyntaxError):
        return []

# Filtrer les étiquettes selon une liste d'étiquettes sélectionnées
def filter_tags(tags_list, selected_tags):
    return [tag for tag in tags_list if tag in selected_tags] if tags_list else None

# Charger les données traitées
questions_data_treated_path = "C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Project_4/data_questions/BIG_DATA_processed.csv"
data_treated = pd.read_csv(questions_data_treated_path)

# Compter les occurrences des étiquettes après filtrage des étiquettes indésirables
all_tags = data_treated['Tags'].apply(safely_evaluate_list).explode()
tag_counts = Counter(all_tags)

# Générer des configurations pour les étiquettes les plus fréquentes
common_tags_configs = {
    'top_15': set([tag for tag, count in tag_counts.most_common(15)]),
    'top_50': set([tag for tag, count in tag_counts.most_common(50)])
}

# Définir le nombre maximal d'étiquettes pour chaque configuration
max_tags_dict = {
    'top_15': 15,
    'top_50': 50
}

# Initialiser les ensembles de données filtrés
filtered_datasets = {}

# Filtrer les données en fonction des configurations d'étiquettes sélectionnées
for config_name, selected_tags in common_tags_configs.items():
    data_filtered = data_treated.copy()
    data_filtered['Tags'] = data_filtered['Tags'].apply(safely_evaluate_list)
    data_filtered['FilteredTags'] = data_filtered['Tags'].apply(lambda x: filter_tags(x, selected_tags))
    data_filtered = data_filtered.dropna(subset=['FilteredTags'])
    data_filtered = data_filtered[data_filtered['FilteredTags'].apply(len) > 0]  # Supprimer les lignes sans étiquettes
    filtered_datasets[config_name] = data_filtered

# Définir les grilles de paramètres pour chaque estimateur
param_grids = {
    'RandomForest': {
        'estimator__n_estimators': [100, 200, 300],
        'estimator__max_depth': [None, 10, 20],
        'estimator__min_samples_split': [2, 5, 10],
    },
    'XGBoost': {
        'estimator__n_estimators': [100, 200],
        'estimator__max_depth': [3, 5, 7],
        'estimator__learning_rate': [0.01, 0.1, 0.2],
        'estimator__subsample': [0.8, 1.0],
    },
    'LogisticRegression': {
        'estimator__C': [0.1, 1, 10],
        'estimator__penalty': ['l2'],
        'estimator__solver': ['lbfgs', 'saga'],
    },
    'GradientBoosting': {
        'estimator__n_estimators': [100, 200],
        'estimator__learning_rate': [0.01, 0.1],
        'estimator__max_depth': [3, 5, 7],
    },
    'AdaBoost': {
        'estimator__n_estimators': [50, 100, 200],
        'estimator__learning_rate': [0.01, 0.1, 1],
    }
}

# Définir les estimateurs de base
base_estimators = {
    'RandomForest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'GradientBoosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier()
}


# Fonction pour vérifier si une exécution a déjà été effectuée pour une configuration et une caractéristique spécifiques
def check_previous_run(config_name, estimator_name, feature_name):
    experiment_id = mlflow.get_experiment_by_name("Default").experiment_id
    runs = mlflow.search_runs(experiment_ids=[experiment_id], filter_string=f"tags.config_name = '{config_name}' and tags.estimator_name = '{estimator_name}' and tags.feature_name = '{feature_name}'")
    if not runs.empty:
        # Vérifier si les métriques de précision moyenne et de couverture sont déjà enregistrées
        if 'metrics.Average Accuracy' in runs.columns and 'metrics.Average Coverage' in runs.columns:
            avg_accuracy = runs['metrics.Average Accuracy'].max()
            avg_coverage = runs['metrics.Average Coverage'].max()
            print(f"Exécution déjà existante pour {config_name} avec {estimator_name} sur {feature_name}. Saut de l'exécution...")
            print(f"Précision moyenne précédente : {avg_accuracy}, Couverture moyenne précédente : {avg_coverage}")
            return True  # L'exécution existe déjà
    return False  # L'exécution n'existe pas

# Fonction pour tracer la précision en fonction du nombre d'étiquettes prédites
def plot_accuracies_per_tag_count(accuracies_per_tag_count, max_tags, config_name, feature_name, estimator_name):
    plt.figure(figsize=(10, 6))

    for real_tag_count, accuracy_list in accuracies_per_tag_count.items():
        x_values = list(range(real_tag_count, max_tags + 1))
        y_values = accuracy_list[:len(x_values)]
        plt.plot(x_values, y_values, label=f'{real_tag_count} vraies étiquettes')

    plt.xlabel('Nombre d\'étiquettes prédites')
    plt.ylabel('Précision moyenne')
    plt.title(f'Précision moyenne par nombre d\'étiquettes prédites - {config_name} - {feature_name} - {estimator_name}')
    plt.legend()
    plt.grid(True)

    # Enregistrer le graphique au lieu de l'afficher
    filename = f"accuracy_{config_name}_{feature_name}_{estimator_name}.png"
    save_path = os.path.join("C:\\Users\\pdevi\\OneDrive\\Desktop\\OpenClassrooms\\Project_4\\plots", filename)
    plt.savefig(save_path)
    plt.close()

# Fonction pour tracer la couverture des étiquettes en fonction du nombre d'étiquettes prédites
def plot_coverage_per_num_tags(coverage_progression, max_tags, config_name, feature_name, estimator_name):
    plt.figure(figsize=(10, 6))

    # Calculer la couverture moyenne par nombre d'étiquettes prédites
    x_values = list(coverage_progression.keys())
    y_values = [np.mean(coverage_progression[num_tags]) for num_tags in x_values]

    # Tracer la couverture moyenne
    plt.plot(x_values, y_values, marker='o', label='Couverture moyenne des étiquettes')

    plt.xlabel('Nombre d\'étiquettes prédites')
    plt.ylabel('Couverture moyenne des étiquettes')
    plt.title(f'Couverture moyenne par nombre d\'étiquettes prédites - {config_name} - {feature_name} - {estimator_name}')
    plt.grid(True)
    plt.legend()

    # Enregistrer le graphique au lieu de l'afficher
    filename = f"mean_coverage_{config_name}_{feature_name}_{estimator_name}.png"
    save_path = os.path.join("C:\\Users\\pdevi\\OneDrive\\Desktop\\OpenClassrooms\\Project_4\\plots", filename)
    plt.savefig(save_path)
    plt.close()

    print(f"Graphique sauvegardé à {save_path}")

# Calculer la précision
def calculate_accuracy(real_tags, predicted_tags):
    if len(real_tags) == 0:  # Protection : renvoyer 0 s'il n'y a pas de vraies étiquettes
        return 0
    correct_predictions = 0
    for predicted_tag in predicted_tags:
        if any(predicted_tag in real_tag for real_tag in real_tags):
            correct_predictions += 1
    return min(correct_predictions / len(real_tags), 1)

# Calculer la couverture des étiquettes
def calculate_tag_coverage(real_tags, predicted_tags, covered_tags):
    for real_tag in real_tags:
        if any(real_tag in predicted_tag for predicted_tag in predicted_tags):
            covered_tags.add(real_tag)
    return covered_tags

# Entraîner, évaluer et enregistrer les modèles avec MLflow
def train_and_evaluate(X, y, base_estimator, param_grid, config_name, estimator_name, max_tags, feature_name, mlb, pca):
    global ii

    mlflow.start_run(run_name=f"{estimator_name}_{config_name}_{feature_name}")
    mlflow.set_tag("config_name", config_name)
    mlflow.set_tag("estimator_name", estimator_name)
    mlflow.set_tag("feature_name", feature_name)

    try:
        # Diviser les données en ensembles d'entraînement et de test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12)
        
        # Effectuer la classification One-vs-Rest avec recherche de grille pour l'optimisation des hyperparamètres
        ovr_clf = OneVsRestClassifier(base_estimator)
        grid_search = GridSearchCV(ovr_clf, param_grid=param_grid, scoring='f1_macro', cv=3, n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)
        num_classes = len(mlb.classes_)  # Nombre total d'étiquettes (classes)
    
        # Initialiser le suivi de la couverture : pour chaque étiquette, suivre combien de fois elle a été prédite correctement
        tag_coverage = {tag: [False] * max_tags for tag in mlb.classes_}
        # Obtenir le meilleur classificateur après la recherche de grille
        best_clf = grid_search.best_estimator_
        y_pred_proba = best_clf.predict_proba(X_test)
        mlflow.log_params(grid_search.best_params_)

        # Initialiser les variables de suivi
        accuracies_per_tag_count = {}  # Contiendra les listes de précisions pour chaque nombre d'étiquettes réelles
        coverage_per_num_tags = {i: [] for i in range(1, max_tags + 1)}  # Couverture moyenne pour chaque nombre d'étiquettes prédites

        flat_accuracy_real_tags = []  # Stocker la précision plate lorsque les étiquettes prédites == étiquettes réelles
        flat_coverage_real_tags = []  # Stocker la couverture plate lorsque les étiquettes prédites == étiquettes réelles

        total_flat_accuracy = 0  # Somme des précisions plates sur tous les échantillons de test
        total_entries_flat = 0   # Nombre d'entrées utilisées pour la précision plate

        covered_tags_set = set()  # Suivre toutes les étiquettes couvertes pendant l'évaluation

        # Boucle à travers chaque échantillon de test
        for i in range(len(X_test)):
            real_tags = mlb.inverse_transform(y_test[i].reshape(1, -1))[0]  # Obtenir les vraies étiquettes pour cet échantillon
            sorted_indices = np.argsort(-y_pred_proba[i])  # Trier les étiquettes prédites par probabilité
            num_real_tags = len(real_tags)  # Nombre de vraies étiquettes dans cet échantillon

            # Initialiser les listes pour les précisions et les couvertures par nombre d'étiquettes réelles si elles ne sont pas déjà initialisées
            if num_real_tags not in accuracies_per_tag_count:
                accuracies_per_tag_count[num_real_tags] = []

            # Itérer sur le nombre d'étiquettes prédites, de 1 à max_tags
            for num_tags_to_predict in range(1, max_tags + 1):
                predicted_tags = [mlb.classes_[index] for index in sorted_indices[:num_tags_to_predict]]

                # Calculer la précision et la couverture pour ce nombre d'étiquettes prédites
                accuracy = calculate_accuracy(real_tags, predicted_tags)
                accuracies_per_tag_count[num_real_tags].append(accuracy)

                covered_tags_set = calculate_tag_coverage(real_tags, predicted_tags, covered_tags_set)
                coverage = len(covered_tags_set) / len(mlb.classes_)
                for tag in real_tags:
                    if tag in predicted_tags:
                        tag_coverage[tag][num_tags_to_predict - 1] = True
            
                # Calculer la progression de la couverture des étiquettes pour chaque nombre d'étiquettes prédites

                # Si le nombre d'étiquettes prédites est égal au nombre d'étiquettes réelles, enregistrer la précision plate et la couverture
                if num_tags_to_predict == num_real_tags:
                    flat_accuracy_real_tags.append(accuracy)
                    flat_coverage_real_tags.append(coverage)
        
        # Calculer la couverture moyenne par nombre d'étiquettes prédites
        coverage_progression = {}
        for num_tags in range(1, max_tags + 1):
            num_tags_covered = sum([any(tag_coverage[tag][:num_tags]) for tag in tag_coverage])
            coverage_progression[num_tags] = num_tags_covered / num_classes
        
        # Calculer la précision moyenne plate et la couverture pour prédire exactement le nombre d'étiquettes réelles
        avg_flat_accuracy = np.mean(flat_accuracy_real_tags)  # La méthode précédente
        avg_flat_coverage = np.mean(flat_coverage_real_tags)
        # Calculer la couverture moyenne par nombre d'étiquettes prédites

        # Enregistrer les métriques plates dans MLflow
        mlflow.log_metric('Average Accuracy', avg_flat_accuracy)
        mlflow.log_metric('Average Coverage', avg_flat_coverage)
        # Enregistrer les précisions par nombre d'étiquettes et la couverture par nombre d'étiquettes comme artefacts ou métriques individuelles
        mlflow.log_dict(accuracies_per_tag_count, "accuracies_per_tag_count.json")
        mlflow.log_dict(coverage_progression, "average_progression.json")

        # Enregistrer le modèle PCA et le meilleur classificateur dans MLflow
        mlflow.sklearn.log_model(pca, f"pca_model_{estimator_name}_{config_name}_{feature_name}")
        mlflow.sklearn.log_model(best_clf, f"model_{estimator_name}_{config_name}_{feature_name}")
        joblib.dump(mlb, f"mlb_{estimator_name}_{config_name}_{feature_name}.pkl")
        mlflow.log_artifact(f"mlb_{estimator_name}_{config_name}_{feature_name}.pkl")
        os.remove(f"mlb_{estimator_name}_{config_name}_{feature_name}.pkl")  # Supprimer le fichier local après l'avoir enregistré
        # Retourner les précisions et couvertures par nombre d'étiquettes sous forme de listes au lieu de moyennes
        return avg_flat_accuracy, avg_flat_coverage, accuracies_per_tag_count, coverage_progression, best_clf

    finally:
        mlflow.end_run()

# Sauvegarder le meilleur modèle, PCA et MultiLabelBinarizer
def save_best_model_pca_mlb(pca, best_model, mlb, config_name):
    mlflow.start_run(run_name=f"Best_Model_{config_name}")
    try:
        mlflow.sklearn.log_model(pca, f"pca_model_{config_name}")
        mlflow.sklearn.log_model(best_model, f"best_model_{config_name}")
        joblib.dump(mlb, f"mlb_{config_name}.pkl")
        mlflow.log_artifact(f"mlb_{config_name}.pkl")
        os.remove(f"mlb_{config_name}.pkl")  # Supprimer le fichier local après l'avoir enregistré
    finally:
        mlflow.end_run()

# Fonction pour aligner les matrices de caractéristiques avec les ensembles de données filtrés et supprimer les étiquettes vides
def align_features_with_filtered_data(filtered_datasets, combined_features, original_data):
    aligned_features = {}
    
    for config_name, data_filtered in filtered_datasets.items():
        # Supprimer les lignes avec des étiquettes vides ou manquantes
        data_filtered = data_filtered.dropna(subset=['FilteredTags'])
        data_filtered = data_filtered[data_filtered['FilteredTags'].apply(len) > 0]
        
        # Obtenir les indices de l'ensemble de données filtré par rapport aux données d'origine
        original_indices = data_filtered.index.tolist()
        aligned_features[config_name] = {}

        # Filtrer chaque matrice de caractéristiques en fonction des indices d'origine de l'ensemble de données filtré
        for feature_name, feature_matrix in combined_features.items():
            aligned_features[config_name][feature_name] = feature_matrix[original_indices]
    
    return aligned_features

# Fonction pour appliquer PCA et évaluer les modèles pour chaque type de caractéristique après l'alignement des données
def evaluate_models_for_configs(filtered_datasets, aligned_features, param_grids, base_estimators, max_tags_dict):

    ii = 0
    for config_name, data_filtered in filtered_datasets.items():
        mlb = MultiLabelBinarizer()
        one_hot_encoded_tags = mlb.fit_transform(data_filtered['FilteredTags'])
        
        # Boucle à travers chaque type de caractéristique
        for feature_name, feature_matrix in aligned_features[config_name].items():
            print(f"Évaluation de la configuration : {config_name}")
            print(f"Application de PCA sur les caractéristiques {feature_name} pour la configuration {config_name}")
            
            # Appliquer PCA à la matrice de caractéristiques alignée
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(feature_matrix)

            # Entraîner et évaluer les modèles pour chaque estimateur de base
            for estimator_name, param_grid in param_grids.items():
                base_estimator = base_estimators.get(estimator_name)
                # Vérifier si cette configuration a déjà été exécutée
                print(f"{config_name} avec {estimator_name} sur {feature_name}")
                if check_previous_run(config_name, estimator_name, feature_name):
                    print(f"Passage de {config_name} avec {estimator_name} sur {feature_name} - déjà calculé.")
                    print(" ")
                    continue  # Passer si cette configuration a déjà été exécutée
                    
                accuracy, coverage, accuracies_per_tag_count, coverage_progression, best_clf = train_and_evaluate(
                    X_pca, one_hot_encoded_tags, base_estimator, param_grid, config_name, estimator_name, max_tags_dict[config_name], feature_name, mlb, pca
                ) 

                # Tracer les précisions et couvertures par nombre d'étiquettes
                plot_accuracies_per_tag_count(accuracies_per_tag_count, max_tags_dict[config_name], config_name, feature_name, estimator_name)
                plot_coverage_per_num_tags(coverage_progression, max_tags_dict[config_name], config_name, feature_name, estimator_name)

# Charger les caractéristiques combinées à partir du fichier .npy
save_path = "C:\\Users\\pdevi\\OneDrive\\Desktop\\OpenClassrooms\\Project_4\\embeddings_data\\combined_features.npy"
combined_features = np.load(save_path, allow_pickle=True).item()

# Aligner les caractéristiques avec les ensembles de données filtrés (avec suppression des lignes d'étiquettes vides)
aligned_features = align_features_with_filtered_data(filtered_datasets, combined_features, data_treated)

# Appeler la fonction d'évaluation avec les caractéristiques alignées
evaluate_models_for_configs(filtered_datasets, aligned_features, param_grids, base_estimators, max_tags_dict)



Evaluating configuration: top_15
Applying PCA on BoW features for configuration top_15
top_15 with RandomForest on BoW
Run already exists for top_15 with RandomForest on BoW. Skipping...
Previous Average Accuracy: 0.19517045454545454, Previous Average Coverage: 0.9954545454545455
Skipping top_15 with RandomForest on BoW - already computed.
 
top_15 with XGBoost on BoW
Run already exists for top_15 with XGBoost on BoW. Skipping...
Previous Average Accuracy: 0.19400252525252523, Previous Average Coverage: 0.9954292929292929
Skipping top_15 with XGBoost on BoW - already computed.
 
top_15 with LogisticRegression on BoW
Run already exists for top_15 with LogisticRegression on BoW. Skipping...
Previous Average Accuracy: 0.18753156565656562, Previous Average Coverage: 0.9955050505050504
Skipping top_15 with LogisticRegression on BoW - already computed.
 
top_15 with GradientBoosting on BoW
Run already exists for top_15 with GradientBoosting on BoW. Skipping...
Previous Average Accuracy: 0.21

Ce code ci-dessous cherche les meilleurs modèles entraînés pour deux configurations (top_15 et top_50) dans MLflow. Il récupère les meilleures exécutions en fonction des performances de couverture et de précision, puis copie les fichiers d'artéfacts associés (MLB, PCA, et modèle) vers un dossier spécifique sur l'ordinateur de l'utilisateur.

In [9]:
import os
import shutil
from mlflow.tracking import MlflowClient
import mlflow

# Fonction pour trouver les meilleurs modèles pour chaque configuration
def find_best_model_per_config(top_n=5):
    client = MlflowClient()
    destination_folder = "C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Project_4/API"

    # Itérer sur les deux configurations (top_15 et top_50)
    for config_name in ['top_15', 'top_50']:
        # Logique pour récupérer les meilleures exécutions depuis mlflow (inchangée)
        experiment_id = mlflow.get_experiment_by_name("Default").experiment_id
        runs = mlflow.search_runs(experiment_ids=[experiment_id], filter_string=f"tags.config_name = '{config_name}'")

        if runs.empty:
            print(f"Aucune exécution trouvée pour la configuration {config_name}")
            continue

        runs = runs[['run_id', 'metrics.Average Accuracy', 'metrics.Average Coverage', 'tags.estimator_name', 'tags.feature_name']]
        top_runs = runs.sort_values(by='metrics.Average Coverage', ascending=False).head(top_n)

        best_run = top_runs.loc[top_runs['metrics.Average Accuracy'].idxmax()]
        best_run_id = best_run['run_id']
        estimator_name = best_run['tags.estimator_name'] if 'tags.estimator_name' in best_run else 'unknown_estimator'
        feature_name = best_run['tags.feature_name'] if 'tags.feature_name' in best_run else 'unknown_feature'

        # Imprimer la configuration dans le format demandé
        print(f"Meilleure configuration pour {config_name} : Estimateur : {estimator_name}, Caractéristique : {feature_name}")

        # Obtenir l'URI des artéfacts de la meilleure exécution
        artifact_uri = f"mlruns/0/{best_run_id}/artifacts/"

        # Copier et sauvegarder les fichiers dans le dossier d'artéfacts de la meilleure exécution
        try:
            # Trouver et copier le fichier MLB
            mlb_file = find_file_in_artifacts(artifact_uri, 'mlb', '.pkl')
            if mlb_file:
                mlb_dest_path = os.path.join(destination_folder, f"mlb_{config_name}.pkl")
                shutil.copy(mlb_file, mlb_dest_path)
                print(f"Fichier MLB copié vers : {mlb_dest_path}")
            # Trouver et copier le fichier du modèle PCA
            pca_folder = find_folder_in_artifacts(artifact_uri, 'pca')
            if pca_folder:
                pca_model_path = os.path.join(pca_folder, 'model.pkl')
                if os.path.exists(pca_model_path):
                    pca_dest_path = os.path.join(destination_folder, f"pca_{config_name}.pkl")
                    shutil.copy(pca_model_path, pca_dest_path)
                    print(f"Fichier du modèle PCA copié vers : {pca_dest_path}")

            # Trouver et copier le fichier du modèle principal
            model_folder = find_folder_in_artifacts(artifact_uri, 'model')
            if model_folder:
                model_file_path = os.path.join(model_folder, 'model.pkl')
                if os.path.exists(model_file_path):
                    model_dest_path = os.path.join(destination_folder, f"model_{config_name}.pkl")
                    shutil.copy(model_file_path, model_dest_path)
                    print(f"Fichier du modèle copié vers : {model_dest_path}")

        # Gestion des erreurs possibles pendant la copie des artéfacts
        except OSError as e:
            print(f"Erreur lors de la gestion des artéfacts : {e}")

        print(f"Meilleur modèle pour {config_name} sauvegardé avec run_id : {best_run_id}")

# Fonction utilitaire pour trouver un fichier spécifique dans les artéfacts
def find_file_in_artifacts(artifact_uri, file_prefix, file_suffix):
    for root, dirs, files in os.walk(artifact_uri):
        for file in files:
            if file.startswith(file_prefix) and file.endswith(file_suffix):
                return os.path.join(root, file)
    return None

# Fonction utilitaire pour trouver un dossier spécifique dans les artéfacts
def find_folder_in_artifacts(artifact_uri, folder_prefix):
    for root, dirs, files in os.walk(artifact_uri):
        for dir_name in dirs:
            if folder_prefix in dir_name:
                return os.path.join(root, dir_name)
    return None

# Appeler la fonction après exécution du reste du code
find_best_model_per_config()



Meilleure configuration pour top_15 : Estimateur : AdaBoost, Caractéristique : TF-IDF
Fichier MLB copié vers : C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Project_4/API\mlb_top_15.pkl
Fichier du modèle PCA copié vers : C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Project_4/API\pca_top_15.pkl
Fichier du modèle copié vers : C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Project_4/API\model_top_15.pkl
Meilleur modèle pour top_15 sauvegardé avec run_id : 7099d2fe24fa4dd0b2fcdfaac79669aa
Meilleure configuration pour top_50 : Estimateur : AdaBoost, Caractéristique : TF-IDF
Fichier MLB copié vers : C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Project_4/API\mlb_top_50.pkl
Fichier du modèle PCA copié vers : C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Project_4/API\pca_top_50.pkl
Fichier du modèle copié vers : C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Project_4/API\model_top_50.pkl
Meilleur modèle pour top_50 sauvegardé avec run_id : 946cd3a0a7334452b5cbf8857430d2a2


Ce code ci-dessous récupère toutes les exécutions d'un projet MLflow, collecte les métriques importantes (comme la précision moyenne et la couverture moyenne) ainsi que les informations de configuration (nom de la configuration, nom de la caractéristique, nom de l'estimateur), puis enregistre ces informations dans un fichier CSV.

In [2]:
import pandas as pd
from mlflow.tracking import MlflowClient
import mlflow

# Fonction pour sauvegarder les métriques de toutes les exécutions dans un fichier CSV
def save_all_run_metrics_to_csv(output_csv='all_run_metrics.csv'):
    client = MlflowClient()  # Initialiser le client MLflow
    experiment_id = mlflow.get_experiment_by_name("Default").experiment_id  # Récupérer l'ID de l'expérience

    # Initialiser une liste pour stocker les données des exécutions
    run_data = []

    # Récupérer toutes les exécutions de l'expérience
    runs = mlflow.search_runs(experiment_ids=[experiment_id])

    # Itérer à travers chaque exécution et collecter les métriques et tags pertinents
    for index, run in runs.iterrows():
        config_name = run['tags.config_name']  # Récupérer le nom de la configuration
        feature_name = run['tags.feature_name']  # Récupérer le nom de la caractéristique
        estimator_name = run['tags.estimator_name']  # Récupérer le nom de l'estimateur
        average_accuracy = run.get('metrics.Average Accuracy', None)  # Récupérer la précision moyenne
        average_coverage = run.get('metrics.Average Coverage', None)  # Récupérer la couverture moyenne

        # Stocker les données pertinentes dans un dictionnaire
        run_data.append({
            'Config Name': config_name,
            'Feature Name': feature_name,
            'Estimator Name': estimator_name,
            'Average Accuracy': average_accuracy,
            'Average Coverage': average_coverage
        })

    # Convertir la liste de dictionnaires en DataFrame
    df = pd.DataFrame(run_data)

    # Sauvegarder le DataFrame dans un fichier CSV
    df.to_csv(output_csv, index=False)
    print(f"Fichier CSV sauvegardé dans {output_csv}")

# Après l'entraînement et l'évaluation des modèles, appeler cette fonction pour sauvegarder les données
save_all_run_metrics_to_csv()


C:\Users\pdevi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Fichier CSV sauvegardé dans all_run_metrics.csv
